# Attention UNET (Gate Based Attention)

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# ------------------------------------------------------------
# Attention Gate Module (Gate based attention)
# ------------------------------------------------------------
class AttentionGate(nn.Module):
    def __init__(self, F_g, F_l, F_int):
        """
        Args:
            F_g: Number of channels in the gating signal (from decoder).
            F_l: Number of channels in the skip connection (from encoder).
            F_int: Number of intermediate channels.
        """
        super(AttentionGate, self).__init__()
        # Theta_x: 1x1 conv to reduce channels of the encoder feature map
        self.W_x = nn.Conv2d(F_l, F_int, kernel_size=1, stride=1, padding=0, bias=True)
        # Phi_g: 1x1 conv to reduce channels of the gating signal
        self.W_g = nn.Conv2d(F_g, F_int, kernel_size=1, stride=1, padding=0, bias=True)
        # psi: 1x1 conv to produce a single channel attention coefficient map
        self.psi = nn.Conv2d(F_int, 1, kernel_size=1, stride=1, padding=0, bias=True)
        self.relu = nn.ReLU(inplace=True)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x, g):
        """
        Args:
            x: Encoder feature map (skip connection) [B, F_l, H, W]
            g: Gating signal from decoder [B, F_g, H_g, W_g]
        Returns:
            Attention-modulated feature map (same size as x)
        """
        # Apply the 1x1 convolutions
        theta_x = self.W_x(x)  # [B, F_int, H, W]
        phi_g   = self.W_g(g)  # [B, F_int, H_g, W_g]
        # Upsample the gating signal to the size of x if needed
        if phi_g.shape[-2:] != theta_x.shape[-2:]:
            phi_g = F.interpolate(phi_g, size=theta_x.shape[-2:], mode='bilinear', align_corners=True)
        # Sum, apply ReLU, then 1x1 conv and sigmoid to get attention coefficients
        f = self.relu(theta_x + phi_g)
        psi = self.sigmoid(self.psi(f))
        # Multiply attention coefficients with the original encoder feature map
        return x * psi

# ------------------------------------------------------------
# Define the original UNet conv_block (no BN)
# ------------------------------------------------------------
class UNetConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UNetConvBlock, self).__init__()
        self.block = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )
    def forward(self, x):
        return self.block(x)

# ------------------------------------------------------------
# Attention U-Net: UNet with Gate-based Attention on Skip Connections
# ------------------------------------------------------------
class UNet_AT(nn.Module):
    def __init__(self, in_channels=3, out_channels=3):
        super(UNet_AT, self).__init__()
        # -------------------- Encoder --------------------
        self.enc1 = UNetConvBlock(in_channels, 64)
        self.enc2 = UNetConvBlock(64, 128)
        self.enc3 = UNetConvBlock(128, 256)
        self.enc4 = UNetConvBlock(256, 512)
        
        # -------------------- Bridge --------------------
        self.bridge = UNetConvBlock(512, 1024)
        
        # -------------------- Decoder --------------------
        self.up4 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.dec4 = UNetConvBlock(512 + 512, 512)  # after attention gating

        self.up3 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.dec3 = UNetConvBlock(256 + 256, 256)

        self.up2 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.dec2 = UNetConvBlock(128 + 128, 128)

        self.up1 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.dec1 = UNetConvBlock(64 + 64, 64)
        
        # Final 1x1 convolution for output
        self.out_conv = nn.Conv2d(64, out_channels, kernel_size=1)

        # -------------------- Attention Gates --------------------
        # For skip connections at each decoder stage:
        self.attn4 = AttentionGate(F_g=512, F_l=512, F_int=256)
        self.attn3 = AttentionGate(F_g=256, F_l=256, F_int=128)
        self.attn2 = AttentionGate(F_g=128, F_l=128, F_int=64)
        self.attn1 = AttentionGate(F_g=64, F_l=64, F_int=32)

    def forward(self, x):
        # -------------------- Encoder --------------------
        enc1 = self.enc1(x)                      # (B,64,H,W)
        enc2 = self.enc2(F.max_pool2d(enc1, 2))   # (B,128,H/2,W/2)
        enc3 = self.enc3(F.max_pool2d(enc2, 2))   # (B,256,H/4,W/4)
        enc4 = self.enc4(F.max_pool2d(enc3, 2))   # (B,512,H/8,W/8)
        
        # -------------------- Bridge --------------------
        bridge = self.bridge(F.max_pool2d(enc4, 2))  # (B,1024,H/16,W/16)
        
        # -------------------- Decoder --------------------
        # Up4: upsample bridge and gate encoder feature enc4
        d4 = self.up4(bridge)  # (B,512,H/8,W/8)
        # Apply attention gate to encoder output enc4 using d4 as gating signal
        attn_enc4 = self.attn4(enc4, d4)
        d4_cat = torch.cat([attn_enc4, d4], dim=1)  # (B,512+512,H/8,W/8)
        d4 = self.dec4(d4_cat)  # (B,512,H/8,W/8)

        # Up3: upsample d4 and gate encoder output enc3
        d3 = self.up3(d4)  # (B,256,H/4,W/4)
        attn_enc3 = self.attn3(enc3, d3)
        d3_cat = torch.cat([attn_enc3, d3], dim=1)  # (B,256+256,H/4,W/4)
        d3 = self.dec3(d3_cat)  # (B,256,H/4,W/4)

        # Up2: upsample d3 and gate encoder output enc2
        d2 = self.up2(d3)  # (B,128,H/2,W/2)
        attn_enc2 = self.attn2(enc2, d2)
        d2_cat = torch.cat([attn_enc2, d2], dim=1)  # (B,128+128,H/2,W/2)
        d2 = self.dec2(d2_cat)  # (B,128,H/2,W/2)

        # Up1: upsample d2 and gate encoder output enc1
        d1 = self.up1(d2)  # (B,64,H,W)
        attn_enc1 = self.attn1(enc1, d1)
        d1_cat = torch.cat([attn_enc1, d1], dim=1)  # (B,64+64,H,W)
        d1 = self.dec1(d1_cat)  # (B,64,H,W)

        # Final output
        out = self.out_conv(d1)  # (B,out_channels,H,W)
        return out


# SE (Squeeze-and-Excitation) Attention

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# ------------------------------------------------------------
# SE Block (Squeeze-and-Excitation)
# ------------------------------------------------------------
class SEBlock(nn.Module):
    def __init__(self, channel, reduction=16):
        """
        Args:
            channel (int): Number of input channels.
            reduction (int): Reduction ratio for the intermediate layer.
        """
        super(SEBlock, self).__init__()
        self.fc1 = nn.Linear(channel, channel // reduction, bias=True)
        self.fc2 = nn.Linear(channel // reduction, channel, bias=True)
        self.relu = nn.ReLU(inplace=True)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        # x: (B, C, H, W)
        b, c, h, w = x.size()
        # Squeeze: Global Average Pooling, output shape (B, C)
        y = x.view(b, c, -1).mean(dim=2)
        # Excitation: two FC layers with ReLU and Sigmoid
        y = self.fc1(y)
        y = self.relu(y)
        y = self.fc2(y)
        y = self.sigmoid(y).view(b, c, 1, 1)
        # Scale: multiply attention weights with the original feature map
        return x * y

# ------------------------------------------------------------
# Basic UNet convolutional block (without BN)
# ------------------------------------------------------------
class UNetConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UNetConvBlock, self).__init__()
        self.block = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )
        
    def forward(self, x):
        return self.block(x)

# ------------------------------------------------------------
# UNet with SE Attention (Attention U-Net with SE instead of spatial attention)
# ------------------------------------------------------------
class UNet_SE(nn.Module):
    def __init__(self, in_channels=3, out_channels=3):
        super(UNet_SE, self).__init__()
        
        # -------------------- Encoder --------------------
        self.enc1 = UNetConvBlock(in_channels, 64)
        self.se1 = SEBlock(64)           # SE attention for encoder 1

        self.enc2 = UNetConvBlock(64, 128)
        self.se2 = SEBlock(128)          # SE attention for encoder 2

        self.enc3 = UNetConvBlock(128, 256)
        self.se3 = SEBlock(256)          # SE attention for encoder 3

        self.enc4 = UNetConvBlock(256, 512)
        self.se4 = SEBlock(512)          # SE attention for encoder 4

        # -------------------- Bridge --------------------
        self.bridge = UNetConvBlock(512, 1024)
        self.se_bridge = SEBlock(1024)   # SE attention for bridge

        # -------------------- Decoder --------------------
        self.up4 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.dec4 = UNetConvBlock(512 + 512, 512)  # after concatenating with gated encoder features

        self.up3 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.dec3 = UNetConvBlock(256 + 256, 256)

        self.up2 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.dec2 = UNetConvBlock(128 + 128, 128)

        self.up1 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.dec1 = UNetConvBlock(64 + 64, 64)
        
        # Final 1x1 convolution for output segmentation
        self.out_conv = nn.Conv2d(64, out_channels, kernel_size=1)
    
    def forward(self, x):
        # -------------------- Encoder --------------------
        enc1 = self.enc1(x)       # (B, 64, H, W)
        enc1 = self.se1(enc1)     # Apply SE attention
        
        enc2 = self.enc2(F.max_pool2d(enc1, 2))  # (B, 128, H/2, W/2)
        enc2 = self.se2(enc2)
        
        enc3 = self.enc3(F.max_pool2d(enc2, 2))  # (B, 256, H/4, W/4)
        enc3 = self.se3(enc3)
        
        enc4 = self.enc4(F.max_pool2d(enc3, 2))  # (B, 512, H/8, W/8)
        enc4 = self.se4(enc4)
        
        # -------------------- Bridge --------------------
        bridge = self.bridge(F.max_pool2d(enc4, 2))  # (B, 1024, H/16, W/16)
        bridge = self.se_bridge(bridge)
        
        # -------------------- Decoder --------------------
        d4 = self.up4(bridge)  # (B, 512, H/8, W/8)
        d4 = torch.cat([enc4, d4], dim=1)  # (B, 512+512, H/8, W/8)
        d4 = self.dec4(d4)  # (B, 512, H/8, W/8)
        
        d3 = self.up3(d4)  # (B, 256, H/4, W/4)
        d3 = torch.cat([enc3, d3], dim=1)  # (B, 256+256, H/4, W/4)
        d3 = self.dec3(d3)  # (B, 256, H/4, W/4)
        
        d2 = self.up2(d3)  # (B, 128, H/2, W/2)
        d2 = torch.cat([enc2, d2], dim=1)  # (B, 128+128, H/2, W/2)
        d2 = self.dec2(d2)  # (B, 128, H/2, W/2)
        
        d1 = self.up1(d2)  # (B, 64, H, W)
        d1 = torch.cat([enc1, d1], dim=1)  # (B, 64+64, H, W)
        d1 = self.dec1(d1)  # (B, 64, H, W)
        
        out = self.out_conv(d1)  # (B, out_channels, H, W)
        return out


# CBAM (Convolutional Block Attention Module)

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# --------------------------------------------------------------------------
# CBAM: Convolutional Block Attention Module
# --------------------------------------------------------------------------

class ChannelAttention(nn.Module):
    def __init__(self, in_channels, reduction=16):
        """
        Channel Attention Module.
        
        Args:
            in_channels (int): Number of input channels.
            reduction (int): Reduction ratio for the intermediate channel number.
        """
        super(ChannelAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)  # output size (B, C, 1, 1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)  # output size (B, C, 1, 1)
        
        self.fc1 = nn.Conv2d(in_channels, in_channels // reduction, kernel_size=1, bias=True)
        self.relu = nn.ReLU(inplace=True)
        self.fc2 = nn.Conv2d(in_channels // reduction, in_channels, kernel_size=1, bias=True)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        # x: (B, C, H, W)
        avg_out = self.fc2(self.relu(self.fc1(self.avg_pool(x))))
        max_out = self.fc2(self.relu(self.fc1(self.max_pool(x))))
        out = avg_out + max_out
        return self.sigmoid(out)

class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        """
        Spatial Attention Module.
        
        Args:
            kernel_size (int): Size of the convolution kernel.
        """
        super(SpatialAttention, self).__init__()
        self.conv = nn.Conv2d(2, 1, kernel_size=kernel_size, padding=kernel_size // 2, bias=False)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        # x: (B, C, H, W)
        avg_out = torch.mean(x, dim=1, keepdim=True)  # (B, 1, H, W)
        max_out, _ = torch.max(x, dim=1, keepdim=True)  # (B, 1, H, W)
        x_cat = torch.cat([avg_out, max_out], dim=1)  # (B, 2, H, W)
        attn = self.conv(x_cat)
        return self.sigmoid(attn)

class CBAM(nn.Module):
    def __init__(self, in_channels, reduction=16, kernel_size=7):
        """
        CBAM combines Channel and Spatial Attention.
        
        Args:
            in_channels (int): Number of channels in the input feature map.
            reduction (int): Reduction ratio for Channel Attention.
            kernel_size (int): Kernel size for Spatial Attention.
        """
        super(CBAM, self).__init__()
        self.channel_attention = ChannelAttention(in_channels, reduction)
        self.spatial_attention = SpatialAttention(kernel_size)
        
    def forward(self, x):
        out = x * self.channel_attention(x)
        out = out * self.spatial_attention(out)
        return out

# --------------------------------------------------------------------------
# Basic UNet Convolution Block (without Batch Normalization)
# --------------------------------------------------------------------------
class UNetConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UNetConvBlock, self).__init__()
        self.block = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )
        
    def forward(self, x):
        return self.block(x)

# --------------------------------------------------------------------------
# UNet with CBAM Attention on Skip Connections
# --------------------------------------------------------------------------
class UNet_CBAM(nn.Module):
    def __init__(self, in_channels=3, out_channels=3):
        super(UNet_CBAM, self).__init__()
        # Encoder
        self.enc1 = UNetConvBlock(in_channels, 64)
        self.cbam1 = CBAM(64)
        
        self.enc2 = UNetConvBlock(64, 128)
        self.cbam2 = CBAM(128)
        
        self.enc3 = UNetConvBlock(128, 256)
        self.cbam3 = CBAM(256)
        
        self.enc4 = UNetConvBlock(256, 512)
        self.cbam4 = CBAM(512)
        
        # Bridge
        self.bridge = UNetConvBlock(512, 1024)
        self.cbam_bridge = CBAM(1024)
        
        # Decoder
        self.up4 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.dec4 = UNetConvBlock(512 + 512, 512)
        
        self.up3 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.dec3 = UNetConvBlock(256 + 256, 256)
        
        self.up2 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.dec2 = UNetConvBlock(128 + 128, 128)
        
        self.up1 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.dec1 = UNetConvBlock(64 + 64, 64)
        
        # Final output layer
        self.out_conv = nn.Conv2d(64, out_channels, kernel_size=1)
        
    def forward(self, x):
        # -------------------- Encoder --------------------
        e1 = self.enc1(x)           # (B, 64, H, W)
        e1 = self.cbam1(e1)         # Apply CBAM
        
        e2 = self.enc2(F.max_pool2d(e1, 2))  # (B, 128, H/2, W/2)
        e2 = self.cbam2(e2)
        
        e3 = self.enc3(F.max_pool2d(e2, 2))  # (B, 256, H/4, W/4)
        e3 = self.cbam3(e3)
        
        e4 = self.enc4(F.max_pool2d(e3, 2))  # (B, 512, H/8, W/8)
        e4 = self.cbam4(e4)
        
        # -------------------- Bridge --------------------
        b = self.bridge(F.max_pool2d(e4, 2))  # (B, 1024, H/16, W/16)
        b = self.cbam_bridge(b)
        
        # -------------------- Decoder --------------------
        d4 = self.up4(b)  # (B, 512, H/8, W/8)
        d4 = torch.cat([e4, d4], dim=1)  # (B, 512+512, H/8, W/8)
        d4 = self.dec4(d4)  # (B, 512, H/8, W/8)
        
        d3 = self.up3(d4)  # (B, 256, H/4, W/4)
        d3 = torch.cat([e3, d3], dim=1)  # (B, 256+256, H/4, W/4)
        d3 = self.dec3(d3)  # (B, 256, H/4, W/4)
        
        d2 = self.up2(d3)  # (B, 128, H/2, W/2)
        d2 = torch.cat([e2, d2], dim=1)  # (B, 128+128, H/2, W/2)
        d2 = self.dec2(d2)  # (B, 128, H/2, W/2)
        
        d1 = self.up1(d2)  # (B, 64, H, W)
        d1 = torch.cat([e1, d1], dim=1)  # (B, 64+64, H, W)
        d1 = self.dec1(d1)  # (B, 64, H, W)
        
        out = self.out_conv(d1)  # (B, out_channels, H, W)
        return out

# Transformer-based Attention (Swin Transformer)

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# --------------------------------------------------------------------------
# Simplified Transformer-based Attention Module (Global Attention)
# --------------------------------------------------------------------------
def window_partition(x, window_size):
    """
    Partition input tensor into non-overlapping windows.
    Args:
        x: (B, C, H, W)
        window_size: int, size of the window (assumes H and W are divisible by window_size)
    Returns:
        windows: (num_windows*B, window_size*window_size, C)
    """
    B, C, H, W = x.shape
    x = x.view(B, C, H // window_size, window_size, W // window_size, window_size)
    # Rearrange to (B, num_windows_H, num_windows_W, window_size, window_size, C)
    x = x.permute(0, 2, 4, 3, 5, 1).contiguous()
    windows = x.view(-1, window_size * window_size, C)
    return windows

def window_reverse(windows, window_size, H, W):
    """
    Reverse the window partitioning operation.
    Args:
        windows: (num_windows*B, window_size*window_size, C)
        window_size: int
        H, W: original height and width
    Returns:
        x: (B, C, H, W)
    """
    B = int(windows.shape[0] / ((H // window_size) * (W // window_size)))
    x = windows.view(B, H // window_size, W // window_size, window_size, window_size, -1)
    x = x.permute(0, 5, 1, 3, 2, 4).contiguous()
    x = x.view(B, -1, H, W)
    return x

# --------------------------------------------------------------------------
# Updated SwinAttention Module with Window Partitioning
# --------------------------------------------------------------------------
class SwinAttention(nn.Module):
    def __init__(self, dim, num_heads=4, mlp_ratio=4.0, dropout=0.0, window_size=8):
        """
        A simplified transformer-based attention block using local (windowed) attention.
        
        Args:
            dim (int): Number of input channels.
            num_heads (int): Number of attention heads.
            mlp_ratio (float): Ratio for the MLP hidden dimension.
            dropout (float): Dropout rate.
            window_size (int): Size of the local attention window.
        """
        super(SwinAttention, self).__init__()
        self.dim = dim
        self.num_heads = num_heads
        self.head_dim = dim // num_heads
        self.scale = self.head_dim ** -0.5
        self.window_size = window_size

        self.qkv = nn.Linear(dim, dim * 3, bias=True)
        self.attn_drop = nn.Dropout(dropout)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(dropout)

        # These LayerNorms expect input of shape (B, L, C) where L is the number of tokens per window.
        self.norm1 = nn.LayerNorm(dim)
        self.norm2 = nn.LayerNorm(dim)
        self.mlp = nn.Sequential(
            nn.Linear(dim, int(dim * mlp_ratio)),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(int(dim * mlp_ratio), dim),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        # x: (B, C, H, W)
        B, C, H, W = x.shape
        # Partition the feature map into windows
        x_windows = window_partition(x, self.window_size)  # (num_windows*B, window_size*window_size, C)
        
        # Apply LayerNorm to each window (treating window tokens as a sequence)
        x_windows_norm = self.norm1(x_windows)  # (num_windows*B, L, C), where L = window_size*window_size
        
        # Compute Q, K, V for each token in each window
        qkv = self.qkv(x_windows_norm)  # (num_windows*B, L, 3 * C)
        L = x_windows_norm.shape[1]  # number of tokens per window
        qkv = qkv.reshape(x_windows_norm.shape[0], L, 3, self.num_heads, self.head_dim)
        qkv = qkv.permute(2, 0, 3, 1, 4)  # (3, num_windows*B, num_heads, L, head_dim)
        q, k, v = qkv[0], qkv[1], qkv[2]  # each: (num_windows*B, num_heads, L, head_dim)
        
        # Scaled dot-product attention within each window
        attn = (q @ k.transpose(-2, -1)) * self.scale  # (num_windows*B, num_heads, L, L)
        attn = attn.softmax(dim=-1)
        attn = self.attn_drop(attn)
        x_attn = attn @ v  # (num_windows*B, num_heads, L, head_dim)
        x_attn = x_attn.transpose(1, 2).reshape(x_windows.shape[0], L, C)  # (num_windows*B, L, C)
        x_attn = self.proj(x_attn)
        x_attn = self.proj_drop(x_attn)
        
        # Residual connection and MLP
        x_out = x_windows + x_attn
        x_out = x_out + self.mlp(self.norm2(x_out))
        
        # Reverse windows to reconstruct the full feature map
        x_out = window_reverse(x_out, self.window_size, H, W)  # (B, C, H, W)
        return x_out

# --------------------------------------------------------------------------
# Basic UNet convolutional block (without BatchNorm)
# --------------------------------------------------------------------------
class UNetConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UNetConvBlock, self).__init__()
        self.block = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )
        
    def forward(self, x):
        return self.block(x)

# --------------------------------------------------------------------------
# UNet with Transformer-based (Swin-inspired) Attention on Skip Connections
# --------------------------------------------------------------------------
class UNet_ST(nn.Module):
    def __init__(self, in_channels=3, out_channels=3):
        super(UNet_ST, self).__init__()
        # Encoder
        self.enc1 = UNetConvBlock(in_channels, 64)
        self.ta1 = SwinAttention(dim=64, num_heads=2, mlp_ratio=4.0)
        
        self.enc2 = UNetConvBlock(64, 128)
        self.ta2 = SwinAttention(dim=128, num_heads=2, mlp_ratio=4.0)
        
        self.enc3 = UNetConvBlock(128, 256)
        self.ta3 = SwinAttention(dim=256, num_heads=4, mlp_ratio=4.0)
        
        self.enc4 = UNetConvBlock(256, 512)
        self.ta4 = SwinAttention(dim=512, num_heads=4, mlp_ratio=4.0)
        
        # Bridge
        self.bridge = UNetConvBlock(512, 1024)
        self.ta_bridge = SwinAttention(dim=1024, num_heads=8, mlp_ratio=4.0)
        
        # Decoder
        self.up4 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.dec4 = UNetConvBlock(512 + 512, 512)
        
        self.up3 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.dec3 = UNetConvBlock(256 + 256, 256)
        
        self.up2 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.dec2 = UNetConvBlock(128 + 128, 128)
        
        self.up1 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.dec1 = UNetConvBlock(64 + 64, 64)
        
        self.out_conv = nn.Conv2d(64, out_channels, kernel_size=1)
    
    def forward(self, x):
        # Encoder
        e1 = self.enc1(x)           # (B,64,H,W)
        e1 = self.ta1(e1)
        
        e2 = self.enc2(F.max_pool2d(e1, 2))  # (B,128,H/2,W/2)
        e2 = self.ta2(e2)
        
        e3 = self.enc3(F.max_pool2d(e2, 2))  # (B,256,H/4,W/4)
        e3 = self.ta3(e3)
        
        e4 = self.enc4(F.max_pool2d(e3, 2))  # (B,512,H/8,W/8)
        e4 = self.ta4(e4)
        
        # Bridge
        b = self.bridge(F.max_pool2d(e4, 2))  # (B,1024,H/16,W/16)
        b = self.ta_bridge(b)
        
        # Decoder
        d4 = self.up4(b)                 # (B,512,H/8,W/8)
        d4 = torch.cat([e4, d4], dim=1)   # (B,1024,H/8,W/8)
        d4 = self.dec4(d4)               # (B,512,H/8,W/8)
        
        d3 = self.up3(d4)                # (B,256,H/4,W/4)
        d3 = torch.cat([e3, d3], dim=1)   # (B,512,H/4,W/4)
        d3 = self.dec3(d3)               # (B,256,H/4,W/4)
        
        d2 = self.up2(d3)                # (B,128,H/2,W/2)
        d2 = torch.cat([e2, d2], dim=1)   # (B,256,H/2,W/2)
        d2 = self.dec2(d2)               # (B,128,H/2,W/2)
        
        d1 = self.up1(d2)                # (B,64,H,W)
        d1 = torch.cat([e1, d1], dim=1)   # (B,128,H,W)
        d1 = self.dec1(d1)               # (B,64,H,W)
        
        out = self.out_conv(d1)          # (B,out_channels,H,W)
        return out

# SCSE (Spatial & Channel Squeeze-and-Excitation)

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# --------------------------------------------------------------------------
# Channel Squeeze & Excitation (cSE)
# --------------------------------------------------------------------------
class cSE(nn.Module):
    def __init__(self, channels, reduction=16):
        super(cSE, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(channels, channels // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channels // reduction, channels, bias=False),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        # x shape: (B, C, H, W)
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1)
        return x * y

# --------------------------------------------------------------------------
# Spatial Squeeze & Excitation (sSE)
# --------------------------------------------------------------------------
class sSE(nn.Module):
    def __init__(self, channels):
        super(sSE, self).__init__()
        self.conv = nn.Conv2d(channels, 1, kernel_size=1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        # x shape: (B, C, H, W)
        y = self.conv(x)
        y = self.sigmoid(y)
        return x * y

# --------------------------------------------------------------------------
# SCSE Block: Concurrent Spatial and Channel Squeeze & Excitation
# --------------------------------------------------------------------------
class SCSE(nn.Module):
    def __init__(self, channels, reduction=16):
        super(SCSE, self).__init__()
        self.cSE = cSE(channels, reduction)
        self.sSE = sSE(channels)
        
    def forward(self, x):
        # Combine the two attentions by element-wise addition.
        return self.cSE(x) + self.sSE(x)

# --------------------------------------------------------------------------
# Basic UNet Convolution Block (without Batch Normalization)
# --------------------------------------------------------------------------
class UNetConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UNetConvBlock, self).__init__()
        self.block = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )
        
    def forward(self, x):
        return self.block(x)

# --------------------------------------------------------------------------
# UNet with SCSE Attention
# --------------------------------------------------------------------------
class UNet_SCSE(nn.Module):
    def __init__(self, in_channels=3, out_channels=3):
        super(UNet_SCSE, self).__init__()
        # Encoder
        self.enc1 = UNetConvBlock(in_channels, 64)
        self.scse1 = SCSE(64)
        
        self.enc2 = UNetConvBlock(64, 128)
        self.scse2 = SCSE(128)
        
        self.enc3 = UNetConvBlock(128, 256)
        self.scse3 = SCSE(256)
        
        self.enc4 = UNetConvBlock(256, 512)
        self.scse4 = SCSE(512)
        
        # Bridge
        self.bridge = UNetConvBlock(512, 1024)
        self.scse_bridge = SCSE(1024)
        
        # Decoder
        self.up4 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.dec4 = UNetConvBlock(512 + 512, 512)
        
        self.up3 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.dec3 = UNetConvBlock(256 + 256, 256)
        
        self.up2 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.dec2 = UNetConvBlock(128 + 128, 128)
        
        self.up1 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.dec1 = UNetConvBlock(64 + 64, 64)
        
        # Final output layer
        self.out_conv = nn.Conv2d(64, out_channels, kernel_size=1)
    
    def forward(self, x):
        # -------------------- Encoder --------------------
        e1 = self.enc1(x)            # (B, 64, H, W)
        e1 = self.scse1(e1)          # Apply SCSE
        
        e2 = self.enc2(F.max_pool2d(e1, 2))  # (B, 128, H/2, W/2)
        e2 = self.scse2(e2)
        
        e3 = self.enc3(F.max_pool2d(e2, 2))  # (B, 256, H/4, W/4)
        e3 = self.scse3(e3)
        
        e4 = self.enc4(F.max_pool2d(e3, 2))  # (B, 512, H/8, W/8)
        e4 = self.scse4(e4)
        
        # -------------------- Bridge --------------------
        b = self.bridge(F.max_pool2d(e4, 2))  # (B, 1024, H/16, W/16)
        b = self.scse_bridge(b)
        
        # -------------------- Decoder --------------------
        d4 = self.up4(b)  # (B, 512, H/8, W/8)
        d4 = torch.cat([e4, d4], dim=1)  # (B, 512+512, H/8, W/8)
        d4 = self.dec4(d4)  # (B, 512, H/8, W/8)
        
        d3 = self.up3(d4)  # (B, 256, H/4, W/4)
        d3 = torch.cat([e3, d3], dim=1)  # (B, 256+256, H/4, W/4)
        d3 = self.dec3(d3)  # (B, 256, H/4, W/4)
        
        d2 = self.up2(d3)  # (B, 128, H/2, W/2)
        d2 = torch.cat([e2, d2], dim=1)  # (B, 128+128, H/2, W/2)
        d2 = self.dec2(d2)  # (B, 128, H/2, W/2)
        
        d1 = self.up1(d2)  # (B, 64, H, W)
        d1 = torch.cat([e1, d1], dim=1)  # (B, 64+64, H, W)
        d1 = self.dec1(d1)  # (B, 64, H, W)
        
        out = self.out_conv(d1)  # (B, out_channels, H, W)
        return out


# Dual Attention (DANet)

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# ------------------------------------------------------------------------------
# Updated Position Attention Module (PAM) with Downsampling
# ------------------------------------------------------------------------------
class PAM(nn.Module):
    def __init__(self, in_channels, downsample_factor=8):
        """
        Position Attention Module with optional downsampling.
        
        Args:
            in_channels (int): Number of input channels.
            downsample_factor (int): Factor to downsample spatially before computing attention.
                                      For example, with factor=8, a (B, C, 640, 640) tensor becomes
                                      (B, C, 80, 80) for the attention computation.
        """
        super(PAM, self).__init__()
        self.downsample_factor = downsample_factor
        if downsample_factor > 1:
            self.pool = nn.MaxPool2d(downsample_factor)
            self.upsample = nn.Upsample(scale_factor=downsample_factor, mode='bilinear', align_corners=True)
        else:
            self.pool = None
            self.upsample = None

        self.query_conv = nn.Conv2d(in_channels, in_channels // 8, kernel_size=1)
        self.key_conv   = nn.Conv2d(in_channels, in_channels // 8, kernel_size=1)
        self.value_conv = nn.Conv2d(in_channels, in_channels, kernel_size=1)
        self.gamma      = nn.Parameter(torch.zeros(1))
        self.softmax    = nn.Softmax(dim=-1)
        
    def forward(self, x):
        # x: (B, C, H, W)
        if self.pool is not None:
            x_down = self.pool(x)  # downsampled input
        else:
            x_down = x
        B, C, H, W = x_down.size()
        N = H * W
        
        # Compute projections
        proj_query = self.query_conv(x_down).view(B, -1, N).permute(0, 2, 1)  # (B, N, C//8)
        proj_key   = self.key_conv(x_down).view(B, -1, N)                        # (B, C//8, N)
        energy     = torch.bmm(proj_query, proj_key)                             # (B, N, N)
        attention  = self.softmax(energy)                                         # (B, N, N)
        proj_value = self.value_conv(x_down).view(B, -1, N)                       # (B, C, N)
        out        = torch.bmm(proj_value, attention.permute(0, 2, 1))            # (B, C, N)
        out        = out.view(B, C, H, W)
        
        if self.upsample is not None:
            out = self.upsample(out)  # upsample back to original resolution
        
        out = self.gamma * out + x    # residual connection
        return out

# ------------------------------------------------------------------------------
# Channel Attention Module (CAM) remains the same
# ------------------------------------------------------------------------------
class CAM(nn.Module):
    def __init__(self, in_channels):
        super(CAM, self).__init__()
        self.gamma = nn.Parameter(torch.zeros(1))
        self.softmax = nn.Softmax(dim=-1)
        
    def forward(self, x):
        # x: (B, C, H, W)
        B, C, H, W = x.size()
        proj_query = x.view(B, C, -1)               # (B, C, N)
        proj_key   = x.view(B, C, -1).permute(0, 2, 1)  # (B, N, C)
        energy     = torch.bmm(proj_query, proj_key)    # (B, C, C)
        # For numerical stability
        energy_new = torch.max(energy, dim=-1, keepdim=True)[0].expand_as(energy) - energy
        attention  = self.softmax(energy_new)         # (B, C, C)
        proj_value = x.view(B, C, -1)                  # (B, C, N)
        out        = torch.bmm(attention, proj_value)   # (B, C, N)
        out        = out.view(B, C, H, W)
        out        = self.gamma * out + x
        return out

# ------------------------------------------------------------------------------
# Dual Attention Module (combining PAM and CAM)
# ------------------------------------------------------------------------------
class DualAttention(nn.Module):
    def __init__(self, in_channels, pam_downsample=8):
        """
        Dual Attention combining PAM and CAM.
        
        Args:
            in_channels (int): Number of input channels.
            pam_downsample (int): Downsampling factor for the PAM component.
        """
        super(DualAttention, self).__init__()
        self.pam = PAM(in_channels, downsample_factor=pam_downsample)
        self.cam = CAM(in_channels)
        
    def forward(self, x):
        pam_out = self.pam(x)
        cam_out = self.cam(x)
        return pam_out + cam_out

# ------------------------------------------------------------------------------
# Basic UNet Convolutional Block (without BatchNorm)
# ------------------------------------------------------------------------------
class UNetConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UNetConvBlock, self).__init__()
        self.block = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )
        
    def forward(self, x):
        return self.block(x)

# ------------------------------------------------------------------------------
# UNet with Dual Attention (DANet-style)
# ------------------------------------------------------------------------------
class UNet_DA(nn.Module):
    def __init__(self, in_channels=3, out_channels=3):
        super(UNet_DA, self).__init__()
        # Encoder
        # For very high-resolution features, use a larger downsample factor to keep the attention map small.
        self.enc1 = UNetConvBlock(in_channels, 64)
        self.da1  = DualAttention(64, pam_downsample=8)   # e1: 640x640 -> 80x80 attention map
       
        self.enc2 = UNetConvBlock(64, 128)
        self.da2  = DualAttention(128, pam_downsample=4)   # e2: 320x320 -> 80x80 (or 320/4=80) attention map
       
        self.enc3 = UNetConvBlock(128, 256)
        self.da3  = DualAttention(256, pam_downsample=2)   # e3: 160x160 -> 80x80 attention map
       
        self.enc4 = UNetConvBlock(256, 512)
        self.da4  = DualAttention(512, pam_downsample=1)   # e4: 80x80 -> 80x80 (no downsampling)
       
        # Bridge
        self.bridge    = UNetConvBlock(512, 1024)
        self.da_bridge = DualAttention(1024, pam_downsample=1)  # Bridge: 40x40 resolution
       
        # Decoder
        self.up4   = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.dec4  = UNetConvBlock(512 + 512, 512)
       
        self.up3   = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.dec3  = UNetConvBlock(256 + 256, 256)
       
        self.up2   = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.dec2  = UNetConvBlock(128 + 128, 128)
       
        self.up1   = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.dec1  = UNetConvBlock(64 + 64, 64)
       
        self.out_conv = nn.Conv2d(64, out_channels, kernel_size=1)
       
    def forward(self, x):
        # Encoder
        e1 = self.enc1(x)
        e1 = self.da1(e1)
       
        e2 = self.enc2(F.max_pool2d(e1, 2))
        e2 = self.da2(e2)
       
        e3 = self.enc3(F.max_pool2d(e2, 2))
        e3 = self.da3(e3)
       
        e4 = self.enc4(F.max_pool2d(e3, 2))
        e4 = self.da4(e4)
       
        # Bridge
        b = self.bridge(F.max_pool2d(e4, 2))
        b = self.da_bridge(b)
       
        # Decoder
        d4 = self.up4(b)
        d4 = torch.cat([e4, d4], dim=1)
        d4 = self.dec4(d4)
       
        d3 = self.up3(d4)
        d3 = torch.cat([e3, d3], dim=1)
        d3 = self.dec3(d3)
       
        d2 = self.up2(d3)
        d2 = torch.cat([e2, d2], dim=1)
        d2 = self.dec2(d2)
       
        d1 = self.up1(d2)
        d1 = torch.cat([e1, d1], dim=1)
        d1 = self.dec1(d1)
       
        out = self.out_conv(d1)
        return out


# Residual Attention Mechanism

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# --------------------------------------------------------------------------
# Residual Attention module
# --------------------------------------------------------------------------
class ResidualAttention(nn.Module):
    def __init__(self, in_channels, reduction=2):
        """
        A simple residual attention module.
        It downsamples the number of channels by 'reduction' factor,
        computes an attention mask, then scales the input features as:
            output = x * (1 + mask)
        """
        super(ResidualAttention, self).__init__()
        intermediate_channels = in_channels // reduction
        self.conv1 = nn.Conv2d(in_channels, intermediate_channels, kernel_size=3, padding=1)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(intermediate_channels, in_channels, kernel_size=3, padding=1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        mask = self.conv1(x)
        mask = self.relu(mask)
        mask = self.conv2(mask)
        mask = self.sigmoid(mask)
        return x * (1 + mask)  # residual attention: emphasize important features

# --------------------------------------------------------------------------
# UNet with Residual Attention (replacing Spatial Attention)
# --------------------------------------------------------------------------
class UNet_RA(nn.Module):
    def __init__(self, in_channels=3, out_channels=3):
        super(UNet_RA, self).__init__()
        
        # Encoder
        self.enc1 = self.conv_block(in_channels, 64)
        self.enc2 = self.conv_block(64, 128)
        self.enc3 = self.conv_block(128, 256)
        self.enc4 = self.conv_block(256, 512)
        
        # Bridge
        self.bridge = self.conv_block(512, 1024)
        
        # Decoder
        self.up4 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.dec4 = self.conv_block(1024, 512)
        
        self.up3 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.dec3 = self.conv_block(512, 256)
        
        self.up2 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.dec2 = self.conv_block(256, 128)
        
        self.up1 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.dec1 = self.conv_block(128, 64)
        
        # Final output layer
        self.out_conv = nn.Conv2d(64, out_channels, kernel_size=1)

        # Replace Spatial Attention modules with Residual Attention modules.
        # The number of channels for each branch is defined by the encoder/bridge.
        self.ra1 = ResidualAttention(64)
        self.ra2 = ResidualAttention(128)
        self.ra3 = ResidualAttention(256)
        self.ra4 = ResidualAttention(512)
        self.ra_bridge = ResidualAttention(1024)

    def conv_block(self, in_channels, out_channels):
        """
        A basic (Conv -> ReLU -> Conv -> ReLU) block, without Batch Normalization.
        """
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )
        
    def forward(self, x):
        # ------------------ Encoder ------------------
        enc1 = self.enc1(x)           # (B,64,H,W)
        enc1 = self.ra1(enc1)         # Apply Residual Attention

        enc2 = self.enc2(F.max_pool2d(enc1, 2))  # (B,128,H/2,W/2)
        enc2 = self.ra2(enc2)

        enc3 = self.enc3(F.max_pool2d(enc2, 2))  # (B,256,H/4,W/4)
        enc3 = self.ra3(enc3)

        enc4 = self.enc4(F.max_pool2d(enc3, 2))  # (B,512,H/8,W/8)
        enc4 = self.ra4(enc4)

        # ------------------ Bridge -------------------
        bridge = self.bridge(F.max_pool2d(enc4, 2))  # (B,1024,H/16,W/16)
        bridge = self.ra_bridge(bridge)

        # ------------------ Decoder ------------------
        # Up4
        dec4 = self.up4(bridge)                  # (B,512,H/8,W/8)
        dec4 = torch.cat([enc4, dec4], dim=1)    # (B,1024,H/8,W/8)
        dec4 = self.dec4(dec4)                   # (B,512,H/8,W/8)

        # Up3
        dec3 = self.up3(dec4)                    # (B,256,H/4,W/4)
        dec3 = torch.cat([enc3, dec3], dim=1)    # (B,512,H/4,W/4)
        dec3 = self.dec3(dec3)                   # (B,256,H/4,W/4)

        # Up2
        dec2 = self.up2(dec3)                    # (B,128,H/2,W/2)
        dec2 = torch.cat([enc2, dec2], dim=1)    # (B,256,H/2,W/2)
        dec2 = self.dec2(dec2)                   # (B,128,H/2,W/2)

        # Up1
        dec1 = self.up1(dec2)                    # (B,64,H,W)
        dec1 = torch.cat([enc1, dec1], dim=1)    # (B,128,H,W)
        dec1 = self.dec1(dec1)                   # (B,64,H,W)

        # Final output layer
        out = self.out_conv(dec1)                # (B,out_channels,H,W)
        return out


In [8]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import pandas as pd
from tqdm import tqdm
from PIL import Image
import gc  # For garbage collection

IMG_HEIGHT = 640
IMG_WIDTH = 640
BATCH_SIZE = 2
EPOCHS = 100
NUM_CLASSES = 3
LEARNING_RATE = 0.001
PATIENCE = 10  # Early stopping
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

IMAGE_DIR = 'CWD-3HSV/train/images'
MASK_DIR  = 'CWD-3HSV/train/Morphed_Images'
VALID_IMAGE_DIR = 'CWD-3HSV/valid/images'
VALID_MASK_DIR  = 'CWD-3HSV/valid/Morphed_Images'

class SegmentationDataset(Dataset):
    def __init__(self, image_files, mask_files, transform=None):
        self.image_files = image_files
        self.mask_files  = mask_files
        self.transform   = transform

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path  = self.image_files[idx]
        mask_path = self.mask_files[idx]
        image = Image.open(img_path).convert('RGB')
        mask  = Image.open(mask_path).convert('L')

        # Resize
        image = image.resize((IMG_WIDTH, IMG_HEIGHT))
        mask  = mask.resize((IMG_WIDTH, IMG_HEIGHT))

        if self.transform:
            image = self.transform(image)
            mask  = torch.tensor(np.array(mask), dtype=torch.long)

        return image, mask

# Prepare training file paths
image_files = [f for f in os.listdir(IMAGE_DIR) if f.endswith('.jpg')]
mask_files  = [f.replace('.jpg', '_morphed.png') for f in image_files]

valid_image_files = []
valid_mask_files  = []
for img_file in image_files:
    mask_file = img_file.replace('.jpg', '_morphed.png')
    if mask_file in os.listdir(MASK_DIR):
        valid_image_files.append(os.path.join(IMAGE_DIR, img_file))
        valid_mask_files.append(os.path.join(MASK_DIR,  mask_file))

val_image_files = [os.path.join(VALID_IMAGE_DIR, f) for f in os.listdir(VALID_IMAGE_DIR) if f.endswith('.jpg')]
val_mask_files  = [os.path.join(VALID_MASK_DIR,  f.replace('.jpg', '_morphed.png'))
                   for f in os.listdir(VALID_IMAGE_DIR) if f.endswith('.jpg')]

transform = transforms.Compose([transforms.ToTensor()])

train_dataset = SegmentationDataset(valid_image_files, valid_mask_files, transform=transform)
val_dataset   = SegmentationDataset(val_image_files,   val_mask_files,   transform=transform)

train_loader  = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader    = DataLoader(val_dataset,   batch_size=BATCH_SIZE, shuffle=False)

def calculate_iou(outputs, masks, num_classes):
    # outputs: (B, num_classes, H, W)
    outputs = torch.argmax(outputs, dim=1)
    iou_per_class = []
    for cls in range(num_classes):
        intersection = ((outputs == cls) & (masks == cls)).sum().item()
        union        = ((outputs == cls) | (masks == cls)).sum().item()
        if union == 0:
            iou_per_class.append(float('nan'))
        else:
            iou_per_class.append(intersection / union)
    return np.nanmean(iou_per_class)

def calculate_iou_loss(outputs, masks, num_classes):
    # 1 - mean IoU
    iou = calculate_iou(outputs, masks, num_classes)
    return 1 - iou

def train_epoch(model, data_loader, optimizer, criterion):
    model.train()
    running_loss     = 0.0
    running_iou_loss = 0.0
    correct          = 0
    total            = 0
    iou_score        = 0

    for images, masks in tqdm(data_loader, desc="Training", leave=False):
        images, masks = images.to(DEVICE), masks.to(DEVICE)

        optimizer.zero_grad()
        outputs  = model(images)
        loss     = criterion(outputs, masks)
        iou_loss = calculate_iou_loss(outputs, masks, NUM_CLASSES)

        loss.backward()
        optimizer.step()

        running_loss     += loss.item()
        running_iou_loss += iou_loss
        _, predicted     = torch.max(outputs, 1)
        total           += masks.numel()
        correct         += (predicted == masks).sum().item()
        iou_score       += calculate_iou(outputs, masks, NUM_CLASSES)

    epoch_loss      = running_loss / len(data_loader)
    epoch_iou_loss  = running_iou_loss / len(data_loader)
    epoch_accuracy  = correct / total * 100
    epoch_iou       = iou_score / len(data_loader)
    return epoch_loss, epoch_accuracy, epoch_iou, epoch_iou_loss

def evaluate(model, data_loader, criterion):
    model.eval()
    running_loss     = 0.0
    running_iou_loss = 0.0
    correct          = 0
    total            = 0
    iou_score        = 0

    with torch.no_grad():
        for images, masks in tqdm(data_loader, desc="Validation", leave=False):
            images, masks = images.to(DEVICE), masks.to(DEVICE)

            outputs  = model(images)
            loss     = criterion(outputs, masks)
            iou_loss = calculate_iou_loss(outputs, masks, NUM_CLASSES)

            running_loss     += loss.item()
            running_iou_loss += iou_loss
            _, predicted     = torch.max(outputs, 1)
            total           += masks.numel()
            correct         += (predicted == masks).sum().item()
            iou_score       += calculate_iou(outputs, masks, NUM_CLASSES)

    epoch_loss      = running_loss / len(data_loader)
    epoch_iou_loss  = running_iou_loss / len(data_loader)
    epoch_accuracy  = correct / total * 100
    epoch_iou       = iou_score / len(data_loader)
    return epoch_loss, epoch_accuracy, epoch_iou, epoch_iou_loss

def train_and_evaluate_model(model_name, model_class, 
                             train_loader, val_loader,
                             epochs=EPOCHS, patience=PATIENCE):
    """
    Train a given model with the specified name/class 
    and store best model + metrics in a separate folder.
    """
    # 1) Create directory for this model
    model_dir = model_name
    os.makedirs(model_dir, exist_ok=True)

    # 2) Instantiate model + move to device
    model = model_class(in_channels=3, out_channels=NUM_CLASSES).to(DEVICE)

    # 3) Define optimizer + loss
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    criterion = nn.CrossEntropyLoss()

    best_val_loss   = float('inf')
    best_model_path = None
    patience_counter= 0
    records         = []

    for epoch in range(epochs):
        print(f"\n[{model_name}] Epoch {epoch+1}/{epochs}")

        train_loss, train_accuracy, train_iou, train_iou_loss = train_epoch(model, train_loader, optimizer, criterion)
        print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, "
              f"Train IoU: {train_iou:.4f}, Train IoU Loss: {train_iou_loss:.4f}")

        val_loss, val_accuracy, val_iou, val_iou_loss = evaluate(model, val_loader, criterion)
        print(f"Val Loss:   {val_loss:.4f}, Val Accuracy:   {val_accuracy:.2f}%, "
              f"Val IoU:   {val_iou:.4f}, Val IoU Loss:   {val_iou_loss:.4f}")

        records.append([
            epoch+1, 
            train_loss, 
            train_accuracy, 
            val_loss, 
            val_accuracy, 
            train_iou_loss, 
            train_iou, 
            val_iou_loss, 
            val_iou
        ])

        # Early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model_path = os.path.join(model_dir, "unet_best_model.pth")
            torch.save(model, best_model_path)
            print(f"  [*] Best model saved at {best_model_path}")
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= patience:
            print(f"  [!] Early stopping for {model_name}")
            break

    # 4) Save Training_Metrics.xlsx in model_dir
    excel_path = os.path.join(model_dir, "Training_Metrics.xlsx")
    columns = [
        "Epoch", 
        "Training Loss", 
        "Training Accuracy", 
        "Validation Loss", 
        "Validation Accuracy", 
        "Training IoU loss", 
        "Mean Training IoU", 
        "Validation IoU loss", 
        "Mean Validation IoU"
    ]
    df = pd.DataFrame(records, columns=columns)
    df.to_excel(excel_path, index=False)
    print(f"  Metrics saved to {excel_path}")

    print(f"Done training {model_name}.\n")

    # Free up GPU memory after training this model
    del model
    gc.collect()  # Force garbage collection
    torch.cuda.empty_cache()

if __name__ == "__main__":
    # List of models you want to train
    models_to_train = {
        "Unet-AT"   : UNet_AT,
        "Unet-SE"   : UNet_SE,
        "Unet-CBAM" : UNet_CBAM,
        "Unet-ST"   : UNet_ST,
        "Unet-SCSE" : UNet_SCSE,
        "Unet-DA"   : UNet_DA,
        "Unet-RA"   : UNet_RA,
    }

    for model_name, model_class in models_to_train.items():
        train_and_evaluate_model(model_name, model_class, 
                                 train_loader, val_loader,
                                 epochs=EPOCHS, patience=PATIENCE)
        # Extra precaution: free any residual CUDA memory after each model training
        gc.collect()
        torch.cuda.empty_cache()



[Unet-AT] Epoch 1/100


Train Loss: 0.4329, Train Accuracy: 82.57%, Train IoU: 0.4710, Train IoU Loss: 0.5290


Val Loss:   0.3358, Val Accuracy:   83.30%, Val IoU:   0.5148, Val IoU Loss:   0.4852
  [*] Best model saved at Unet-AT/unet_best_model.pth

[Unet-AT] Epoch 2/100


Train Loss: 0.2369, Train Accuracy: 90.60%, Train IoU: 0.6455, Train IoU Loss: 0.3545


Val Loss:   0.2200, Val Accuracy:   91.34%, Val IoU:   0.7753, Val IoU Loss:   0.2247
  [*] Best model saved at Unet-AT/unet_best_model.pth

[Unet-AT] Epoch 3/100


...

Val Loss:   0.1047, Val Accuracy:   96.09%, Val IoU:   0.8691, Val IoU Loss:   0.1309

[Unet-RA] Epoch 75/100


Train Loss: 0.0881, Train Accuracy: 96.86%, Train IoU: 0.8394, Train IoU Loss: 0.1606


Val Loss:   0.1145, Val Accuracy:   95.47%, Val IoU:   0.8574, Val IoU Loss:   0.1426

[Unet-RA] Epoch 76/100


Train Loss: 0.0879, Train Accuracy: 96.88%, Train IoU: 0.8398, Train IoU Loss: 0.1602


Val Loss:   0.1068, Val Accuracy:   96.01%, Val IoU:   0.8641, Val IoU Loss:   0.1359
  [!] Early stopping for Unet-RA
  Metrics saved to Unet-RA/Training_Metrics.xlsx
Done training Unet-RA.



# Saving Prediction Images Of Each Model

In [9]:
import os
import random
import numpy as np
import torch
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt

import matplotlib
matplotlib.use('Agg')  # Turn off interactive backend (no pop-up windows)

TEST_IMAGE_FOLDER       = 'CWD-3HSV/test/images'
GROUND_TRUTH_MASK_FOLDER= 'CWD-3HSV/test/Morphed_Images'
IMG_HEIGHT              = 640
IMG_WIDTH               = 640
NUM_CLASSES             = 3
DEVICE                  = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


transform = transforms.Compose([
    transforms.Resize((IMG_HEIGHT, IMG_WIDTH)),
    transforms.ToTensor(),
])


def load_full_model(model_path):
    model = torch.load(model_path, map_location=DEVICE)
    model = model.to(DEVICE)
    model.eval()
    return model

def preprocess_image(image_path):
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(DEVICE)
    return image


def load_ground_truth_mask(mask_path):
    mask = Image.open(mask_path).convert('L')
    mask = mask.resize((IMG_WIDTH, IMG_HEIGHT), Image.NEAREST)
    return np.array(mask)


def generate_segmentation_mask(model, image):
    with torch.no_grad():
        output = model(image)        # (B, NUM_CLASSES, H, W)
        pred   = torch.argmax(output, dim=1)
        return pred.squeeze().cpu().numpy()


def visualize_and_save_comparison(
    model, input_image, gt_mask, class_rgb_mapping, input_image_path,
    save_folder, save_predictions=True
):
    # Load the original image for visualization
    original_image = Image.open(input_image_path).convert('RGB')
    original_image = original_image.resize((IMG_WIDTH, IMG_HEIGHT))

    # Generate predicted mask
    pred_mask = generate_segmentation_mask(model, input_image)

    # Map predicted mask to RGB
    rgb_pred_mask = np.zeros((pred_mask.shape[0], pred_mask.shape[1], 3), dtype=np.uint8)
    for class_id, rgb_value in class_rgb_mapping.items():
        rgb_pred_mask[pred_mask == class_id] = rgb_value

    # Map ground truth to RGB
    rgb_gt_mask = np.zeros((gt_mask.shape[0], gt_mask.shape[1], 3), dtype=np.uint8)
    for class_id, rgb_value in class_rgb_mapping.items():
        rgb_gt_mask[gt_mask == class_id] = rgb_value

    # Plot side-by-side
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    axes[0].imshow(original_image)
    axes[0].set_title('Original')
    axes[0].axis('off')

    axes[1].imshow(rgb_gt_mask)
    axes[1].set_title('Ground Truth')
    axes[1].axis('off')

    axes[2].imshow(rgb_pred_mask)
    axes[2].set_title('Predicted')
    axes[2].axis('off')

    fig.tight_layout()

    # Save the figure (no plt.show())
    if save_predictions:
        # For the figure
        fig_filename = os.path.splitext(os.path.basename(input_image_path))[0] + '_compare.png'
        fig_save_path= os.path.join(save_folder, fig_filename)
        fig.savefig(fig_save_path, bbox_inches='tight')

        # For the predicted mask alone
        pred_mask_img = Image.fromarray(rgb_pred_mask)
        pred_filename = os.path.splitext(os.path.basename(input_image_path))[0] + '_predmask.png'
        pred_save_path= os.path.join(save_folder, pred_filename)
        pred_mask_img.save(pred_save_path)

    plt.close(fig)

# -------------------------------------------------------------------
# Class-to-RGB mapping
# -------------------------------------------------------------------
class_rgb_mapping = {
    0: (0, 0, 0),    # black
    1: (0, 255, 0),  # green
    2: (255, 0, 0),  # red
}

# -------------------------------------------------------------------
# MAIN: 
# 1) Find all "Unet-..." directories
# 2) For each, load "unet_best_model.pth"
# 3) Randomly pick 5 test images, generate predictions
# 4) Save side-by-side figure + predicted mask
# -------------------------------------------------------------------
if __name__ == "__main__":
    # Silence any console printing
    # (Here we can reassign print to a no-op if needed)
    def no_op(*args, **kwargs):
        pass
    print = no_op

    # 1) Find directories that start with "Unet-"
    all_dirs = [d for d in os.listdir('.') if os.path.isdir(d) and d.startswith("Unet")]

    # 2) For each directory, load unet_best_model.pth
    # and do random predictions
    test_image_files = [
        f for f in os.listdir(TEST_IMAGE_FOLDER)
        if f.endswith('.jpg') or f.endswith('.jpeg') or f.endswith('.png')
    ]
    # If <5 images exist, use them all
    if len(test_image_files) <= 5:
        selected_images = test_image_files
    else:
        selected_images = random.sample(test_image_files, 5)

    for model_dir in all_dirs:
        model_path = os.path.join(model_dir, "unet_best_model.pth")
        if not os.path.isfile(model_path):
            continue  # skip if no best model in that dir

        # Create a subfolder "Predictions" inside model_dir
        pred_save_folder = os.path.join(model_dir, "Predictions")
        os.makedirs(pred_save_folder, exist_ok=True)

        # Load model
        model = load_full_model(model_path)

        # For each selected image, compare
        for image_file in selected_images:
            input_image_path = os.path.join(TEST_IMAGE_FOLDER, image_file)
            # ground truth
            gt_mask_name = os.path.splitext(image_file)[0] + '_morphed.png'
            gt_mask_path = os.path.join(GROUND_TRUTH_MASK_FOLDER, gt_mask_name)

            input_image = preprocess_image(input_image_path)
            ground_truth_mask = load_ground_truth_mask(gt_mask_path)

            # Visualize and save
            visualize_and_save_comparison(
                model=model,
                input_image=input_image,
                gt_mask=ground_truth_mask,
                class_rgb_mapping=class_rgb_mapping,
                input_image_path=input_image_path,
                save_folder=pred_save_folder,
                save_predictions=True
            )


/tmp/ipykernel_4204/2188687900.py:27: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(model_path, map_location=DEVICE)


# Evaluating Each Model

In [10]:
import os
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import pandas as pd
from tqdm import tqdm
from PIL import Image
import time

from sklearn.metrics import precision_score, recall_score, f1_score
import matplotlib
matplotlib.use('Agg')  # Turn off interactive display
import matplotlib.pyplot as plt
import seaborn as sns


TEST_IMAGES_DIR = 'CWD-3HSV/test/images'
TEST_MASKS_DIR  = 'CWD-3HSV/test/Morphed_Images'
IMG_HEIGHT      = 640
IMG_WIDTH       = 640
NUM_CLASSES     = 3
DEVICE          = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Suppress console output
def no_op(*args, **kwargs):
    pass
print = no_op  # Overwrite default print

transform = transforms.Compose([
    transforms.Resize((IMG_HEIGHT, IMG_WIDTH)),
    transforms.ToTensor(),
])


def load_model(model_path):
    model = torch.load(model_path, map_location=DEVICE)
    model.to(DEVICE)
    model.eval()
    return model

def preprocess_image(image_path):
    image = Image.open(image_path).convert('RGB')
    return transform(image).unsqueeze(0).to(DEVICE)

def preprocess_mask(mask_path):
    mask = Image.open(mask_path).convert('L')
    mask = mask.resize((IMG_WIDTH, IMG_HEIGHT), Image.NEAREST)
    return np.array(mask)


def generate_predictions(model, image):
    with torch.no_grad():
        output = model(image)            # [B, NUM_CLASSES, H, W]
        pred   = torch.argmax(output, 1) # pick class with max logit
        return pred.squeeze().cpu().numpy()


def calculate_iou(pred_mask, gt_mask, num_classes):
    iou_per_class = []
    for cls in range(num_classes):
        intersection = np.logical_and(pred_mask == cls, gt_mask == cls).sum()
        union        = np.logical_or(pred_mask == cls, gt_mask == cls).sum()
        iou = intersection / union if union > 0 else 0
        iou_per_class.append(iou)
    return iou_per_class

def calculate_dice(pred_mask, gt_mask, num_classes):
    dice_per_class = []
    for cls in range(num_classes):
        intersection = np.logical_and(pred_mask == cls, gt_mask == cls).sum()
        denom        = (np.sum(pred_mask == cls) + np.sum(gt_mask == cls))
        dice         = 2.0 * intersection / denom if denom > 0 else 0
        dice_per_class.append(dice)
    return dice_per_class

def calculate_jaccard(pred_mask, gt_mask, num_classes):
    jaccard_per_class = []
    for cls in range(num_classes):
        intersection = np.logical_and(pred_mask == cls, gt_mask == cls).sum()
        union        = np.logical_or(pred_mask == cls, gt_mask == cls).sum()
        jaccard      = intersection / union if union > 0 else 0
        jaccard_per_class.append(jaccard)
    return jaccard_per_class


def plot_confusion_matrix(cm, class_names, save_path, title="Confusion Matrix", fmt='d'):
    """
    'fmt': 'd' for counts, '.2f' for percentages
    Saves the figure to 'save_path'.
    """
    fig, ax = plt.subplots(figsize=(6,6))
    sns.heatmap(cm, annot=True, fmt=fmt, cmap='Blues',
                xticklabels=class_names, yticklabels=class_names)
    ax.set_xlabel("Predicted")
    ax.set_ylabel("Actual")
    ax.set_title(title)
    plt.tight_layout()
    fig.savefig(save_path, bbox_inches='tight')
    plt.close(fig)


def evaluate_model(model, test_images_dir, test_masks_dir, num_classes, results_dir):
    all_pred = []
    all_gt   = []

    iou_per_class       = np.zeros(num_classes)
    dice_per_class      = np.zeros(num_classes)
    jaccard_per_class   = np.zeros(num_classes)
    accuracy_per_class  = np.zeros(num_classes)
    precision_per_class = np.zeros(num_classes)
    recall_per_class    = np.zeros(num_classes)
    f1_per_class        = np.zeros(num_classes)

    image_files  = [img for img in os.listdir(test_images_dir) if img.endswith('.jpg')]
    total_samples= 0

    for img_name in image_files:
        image_path = os.path.join(test_images_dir, img_name)
        mask_name  = img_name.replace('.jpg', '_morphed.png')
        mask_path  = os.path.join(test_masks_dir, mask_name)

        image   = preprocess_image(image_path)
        gt_mask = preprocess_mask(mask_path)
        pred_mask= generate_predictions(model, image)

        # Flatten
        all_pred.extend(pred_mask.flatten())
        all_gt.extend(gt_mask.flatten())

        # iou/dice/jaccard
        iou_sample     = calculate_iou(pred_mask,     gt_mask, num_classes)
        dice_sample    = calculate_dice(pred_mask,    gt_mask, num_classes)
        jaccard_sample = calculate_jaccard(pred_mask, gt_mask, num_classes)

        iou_per_class     += np.array(iou_sample)
        dice_per_class    += np.array(dice_sample)
        jaccard_per_class += np.array(jaccard_sample)

        # per-class metrics
        for cls in range(num_classes):
            tp = np.sum((pred_mask == cls) & (gt_mask == cls))
            fp = np.sum((pred_mask == cls) & (gt_mask != cls))
            fn = np.sum((pred_mask != cls) & (gt_mask == cls))
            total_class_pixels= np.sum(gt_mask == cls)
            accuracy_per_class[cls] += tp / (total_class_pixels + 1e-6)

            precision_per_class[cls]+= tp / (tp + fp + 1e-6)
            recall_per_class[cls]   += tp / (tp + fn + 1e-6)
            f1_per_class[cls]       += 2*tp / (2*tp + fp + fn + 1e-6)

        total_samples += 1

    # Normalize
    accuracy_per_class  /= total_samples
    precision_per_class /= total_samples
    recall_per_class    /= total_samples
    f1_per_class        /= total_samples
    iou_per_class       /= total_samples
    dice_per_class      /= total_samples
    jaccard_per_class   /= total_samples

    mean_dice     = np.mean(dice_per_class)
    mean_jaccard  = np.mean(jaccard_per_class)

    # freq weighted iou
    all_gt_arr = np.array(all_gt)
    class_counts= np.bincount(all_gt_arr, minlength=num_classes)
    frequency_weighted_iou= np.average(iou_per_class, weights=class_counts)

    # overall metrics
    all_pred_arr = np.array(all_pred)
    accuracy = (all_pred_arr == all_gt_arr).sum()/len(all_gt_arr)
    precision= precision_score(all_gt_arr, all_pred_arr, average='weighted', zero_division=1)
    recall   = recall_score(all_gt_arr,    all_pred_arr, average='weighted', zero_division=1)
    f1       = f1_score(all_gt_arr,        all_pred_arr, average='weighted', zero_division=1)

    mean_iou    = np.mean(iou_per_class)
    weighted_iou= np.average(iou_per_class, weights=np.bincount(all_gt_arr))

    # confusion matrix
    confusion_matrix_counts= np.zeros((num_classes, num_classes), dtype=np.int64)
    for gt_val, pr_val in zip(all_gt_arr, all_pred_arr):
        confusion_matrix_counts[gt_val, pr_val]+=1

    confusion_matrix_percent= np.zeros_like(confusion_matrix_counts, dtype=float)
    for r in range(num_classes):
        row_sum = confusion_matrix_counts[r,:].sum()
        if row_sum > 0:
            confusion_matrix_percent[r,:] = (confusion_matrix_counts[r,:]/ row_sum)*100

    class_names= [f"Class {i}" for i in range(num_classes)]

    # Save confusion matrices as PNG
    cm_counts_path  = os.path.join(results_dir, "Confusion_Matrix_Counts.png")
    cm_percent_path = os.path.join(results_dir, "Confusion_Matrix_Percent.png")
    plot_confusion_matrix(confusion_matrix_counts,  class_names, save_path=cm_counts_path,  title="Confusion Matrix (Counts)", fmt='d')
    plot_confusion_matrix(confusion_matrix_percent, class_names, save_path=cm_percent_path, title="Confusion Matrix (Percent)", fmt='.2f')

    return (
        accuracy, accuracy_per_class, precision, precision_per_class,
        recall, recall_per_class, f1, f1_per_class,
        iou_per_class, mean_iou, weighted_iou, frequency_weighted_iou,
        dice_per_class, jaccard_per_class, mean_dice, mean_jaccard
    )

def save_results_to_excel(
    model_name,
    accuracy, accuracy_per_class,
    precision, precision_per_class,
    recall, recall_per_class,
    f1, f1_per_class,
    iou_per_class, mean_iou,
    weighted_iou, frequency_weighted_iou,
    dice_per_class, jaccard_per_class,
    mean_dice, mean_jaccard,
    save_directory
):
    # We'll store the final xlsx in the same directory as the model
    excel_path = os.path.join(save_directory, "Performance_Evaluation_Metrics.xlsx")

    columns = (
        ['Model Name', 'Accuracy']
        + [f'Accuracy Class {i}' for i in range(len(accuracy_per_class))]
        + ['Precision'] + [f'Precision Class {i}' for i in range(len(precision_per_class))]
        + ['Recall'] + [f'Recall Class {i}' for i in range(len(recall_per_class))]
        + ['F1 Score'] + [f'F1 Score Class {i}' for i in range(len(f1_per_class))]
        + [f'IoU Class {i}' for i in range(len(iou_per_class))]
        + ['Mean IoU', 'Weighted IoU', 'Frequency Weighted IoU']
        + [f'Dice Coefficient Class {i}' for i in range(len(dice_per_class))]
        + ['Mean Dice']
        + [f'Jaccard Index Class {i}' for i in range(len(jaccard_per_class))]
        + ['Mean Jaccard']
    )

    new_row = {
        'Model Name': model_name,
        'Accuracy': accuracy,
        **{f'Accuracy Class {i}': acc for i, acc in enumerate(accuracy_per_class)},
        'Precision': precision,
        **{f'Precision Class {i}': prec for i, prec in enumerate(precision_per_class)},
        'Recall': recall,
        **{f'Recall Class {i}': r for i, r in enumerate(recall_per_class)},
        'F1 Score': f1,
        **{f'F1 Score Class {i}': f1c for i, f1c in enumerate(f1_per_class)},
        **{f'IoU Class {i}': iou for i, iou in enumerate(iou_per_class)},
        'Mean IoU': mean_iou,
        'Weighted IoU': weighted_iou,
        'Frequency Weighted IoU': frequency_weighted_iou,
        **{f'Dice Coefficient Class {i}': d for i, d in enumerate(dice_per_class)},
        'Mean Dice': mean_dice,
        **{f'Jaccard Index Class {i}': j for i, j in enumerate(jaccard_per_class)},
        'Mean Jaccard': mean_jaccard,
    }

    new_data = pd.DataFrame([new_row], columns=columns)

    # Overwrite any existing file to store only the last row
    new_data.to_excel(excel_path, index=False, header=True)

if __name__ == "__main__":
    # Suppress console output
    def no_op(*args, **kwargs):
        pass
    print = no_op  # Overwrite default print

    # 1) Find all directories that match "Unet-<backbone>"
    all_dirs = [d for d in os.listdir('.') if os.path.isdir(d) and d.startswith("Unet")]

    for model_dir in all_dirs:
        # 2) Load "unet_best_model.pth" in that directory, if exists
        model_path = os.path.join(model_dir, "unet_best_model.pth")
        if not os.path.isfile(model_path):
            continue  # skip if no best model found

        model = load_model(model_path)

        # 3) Evaluate
        results = evaluate_model(
            model=model,
            test_images_dir=TEST_IMAGES_DIR,
            test_masks_dir=TEST_MASKS_DIR,
            num_classes=NUM_CLASSES,
            results_dir=model_dir  # store confusion matrix PNG in same directory
        )

        # 4) Unpack
        (
            accuracy, accuracy_per_class,
            precision, precision_per_class,
            recall, recall_per_class,
            f1, f1_per_class,
            iou_per_class, mean_iou,
            weighted_iou, frequency_weighted_iou,
            dice_per_class, jaccard_per_class,
            mean_dice, mean_jaccard
        ) = results

        # 5) Save row to "Performance_Evaluation_Metrics.xlsx" in model_dir
        save_results_to_excel(
            model_name=model_dir,
            accuracy=accuracy,
            accuracy_per_class=accuracy_per_class,
            precision=precision,
            precision_per_class=precision_per_class,
            recall=recall,
            recall_per_class=recall_per_class,
            f1=f1,
            f1_per_class=f1_per_class,
            iou_per_class=iou_per_class,
            mean_iou=mean_iou,
            weighted_iou=weighted_iou,
            frequency_weighted_iou=frequency_weighted_iou,
            dice_per_class=dice_per_class,
            jaccard_per_class=jaccard_per_class,
            mean_dice=mean_dice,
            mean_jaccard=mean_jaccard,
            save_directory=model_dir
        )


/tmp/ipykernel_4204/3703382463.py:38: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(model_path, map_location=DEVICE)
/tmp/ipykernel_4204/3703382463.py:38:

# Saving Training Curves Of Each Model

In [11]:
import os
import pandas as pd
import matplotlib
matplotlib.use('Agg')  # So figures don't pop up; they are just saved
import matplotlib.pyplot as plt

def plot_training_curves_for_model(excel_path, output_dir):
    """
    Reads Training_Metrics.xlsx from excel_path and saves four plots in output_dir.
    """
    # Read Excel
    df = pd.read_excel(excel_path)

    # Extract columns
    epochs            = df['Epoch']
    train_loss        = df['Training Loss']
    val_loss          = df['Validation Loss']
    train_acc         = df['Training Accuracy']
    val_acc           = df['Validation Accuracy']
    train_iou_loss    = df['Training IoU loss']
    val_iou_loss      = df['Validation IoU loss']
    mean_train_iou    = df['Mean Training IoU']
    mean_val_iou      = df['Mean Validation IoU']

    # 1) Training vs Validation Loss
    plt.figure(figsize=(8,6))
    plt.plot(epochs, train_loss, label='Training Loss', marker='o')
    plt.plot(epochs, val_loss,   label='Validation Loss', marker='s')
    plt.title('Training vs Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)

    loss_plot_path = os.path.join(output_dir, 'Train_vs_Val_Loss.png')
    plt.savefig(loss_plot_path, bbox_inches='tight')
    plt.close()

    # 2) Training vs Validation Accuracy
    plt.figure(figsize=(8,6))
    plt.plot(epochs, train_acc, label='Training Accuracy', marker='o')
    plt.plot(epochs, val_acc,   label='Validation Accuracy', marker='s')
    plt.title('Training vs Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy (%)')
    plt.legend()
    plt.grid(True)

    acc_plot_path = os.path.join(output_dir, 'Train_vs_Val_Accuracy.png')
    plt.savefig(acc_plot_path, bbox_inches='tight')
    plt.close()

    # 3) Training IoU Loss vs Validation IoU Loss
    plt.figure(figsize=(8,6))
    plt.plot(epochs, train_iou_loss, label='Training IoU Loss', marker='o')
    plt.plot(epochs, val_iou_loss,   label='Validation IoU Loss', marker='s')
    plt.title('Training vs Validation IoU Loss')
    plt.xlabel('Epoch')
    plt.ylabel('IoU Loss')
    plt.legend()
    plt.grid(True)

    iou_loss_plot_path = os.path.join(output_dir, 'Train_vs_Val_IoU_Loss.png')
    plt.savefig(iou_loss_plot_path, bbox_inches='tight')
    plt.close()

    # 4) Mean Training IoU vs Mean Validation IoU
    plt.figure(figsize=(8,6))
    plt.plot(epochs, mean_train_iou, label='Mean Training IoU', marker='o')
    plt.plot(epochs, mean_val_iou,   label='Mean Validation IoU', marker='s')
    plt.title('Mean Training IoU vs Mean Validation IoU')
    plt.xlabel('Epoch')
    plt.ylabel('IoU')
    plt.legend()
    plt.grid(True)

    iou_plot_path = os.path.join(output_dir, 'Mean_Train_vs_Val_IoU.png')
    plt.savefig(iou_plot_path, bbox_inches='tight')
    plt.close()

def main():
    # 1) Find directories named "Unet-..."
    unet_dirs = [d for d in os.listdir('.') if os.path.isdir(d) and d.startswith('Unet')]

    for unet_dir in unet_dirs:
        # 2) The path to the Training_Metrics.xlsx
        excel_path = os.path.join(unet_dir, 'Training_Metrics.xlsx')
        if not os.path.isfile(excel_path):
            continue  # skip if no metrics file

        # 3) Create "Training_Curves" subdir
        curves_dir = os.path.join(unet_dir, 'Training_Curves')
        os.makedirs(curves_dir, exist_ok=True)

        # 4) Generate + save plots
        plot_training_curves_for_model(excel_path, curves_dir)

if __name__ == "__main__":
    main()


# All Models Performance Evaluation Sheet

In [12]:
import os
import pandas as pd

# 1) Automatically discover directories named "Unet-..."
model_directories = [
    d for d in os.listdir('.') 
    if os.path.isdir(d) and d.startswith("Unet")
]

# 2) Each directory's Excel file name
excel_filename = "Performance_Evaluation_Metrics.xlsx"

# 3) Where to save the merged file
results_folder = "Results"
os.makedirs(results_folder, exist_ok=True)
merged_excel_path = os.path.join(results_folder, "All_Models_Performance_Evaluation_Metrics.xlsx")

# 4) Create an empty list to hold the last rows from each subdirectory
merged_rows = []

# 5) Loop over each Unet-<backbone> directory
for model_dir in model_directories:
    excel_path = os.path.join(model_dir, excel_filename)

    # Check if the file exists
    if not os.path.isfile(excel_path):
        print(f"Warning: {excel_path} not found. Skipping.")
        continue

    # Read entire Excel file
    df = pd.read_excel(excel_path)

    if df.empty:
        print(f"Warning: {excel_path} is empty. Skipping.")
        continue

    # Get the last (bottom) row
    last_row = df.iloc[[-1]].copy()
    merged_rows.append(last_row)

# 6) If we have rows, concatenate them; otherwise create empty DataFrame
if len(merged_rows) > 0:
    merged_df = pd.concat(merged_rows, ignore_index=True)
else:
    merged_df = pd.DataFrame()

# 7) Overwrite the final Excel file with these rows
merged_df.to_excel(merged_excel_path, index=False)
print(f"Merged file saved to: {merged_excel_path}")



# Saving Predictions For All Models

In [13]:
import os
import random
import numpy as np
import torch
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt

# ------------------------------------------------------------------------------
# 1) Automatically gather all directories starting with "Unet-"
# ------------------------------------------------------------------------------
all_dirs = [d for d in os.listdir('.') if os.path.isdir(d)]
model_dirs = [d for d in all_dirs if d.startswith("Unet-")]
models_info = [(d, os.path.join(d, "unet_best_model.pth")) for d in model_dirs]

# ------------------------------------------------------------------------------
# 2) Define directories / file paths and hyperparameters
# ------------------------------------------------------------------------------
TEST_IMAGE_FOLDER        = 'CWD-3HSV/test/images/'
GROUND_TRUTH_MASK_FOLDER = 'CWD-3HSV/test/Morphed_Images/'
PREDICTION_SAVE_FOLDER   = 'Predictions'
os.makedirs(PREDICTION_SAVE_FOLDER, exist_ok=True)

IMG_HEIGHT  = 640
IMG_WIDTH   = 640
NUM_CLASSES = 3
DEVICE      = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# ------------------------------------------------------------------------------
# 3) Define transformation (as used during training)
# ------------------------------------------------------------------------------
transform = transforms.Compose([
    transforms.Resize((IMG_HEIGHT, IMG_WIDTH)),
    transforms.ToTensor(),
])

# ------------------------------------------------------------------------------
# 4) Helper functions
# ------------------------------------------------------------------------------
def load_full_model(model_path):
    model = torch.load(model_path, map_location=DEVICE)
    model = model.to(DEVICE)
    model.eval()
    return model

def preprocess_image(image_path):
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(DEVICE)
    return image

def load_ground_truth_mask(mask_path):
    mask = Image.open(mask_path).convert('L')
    mask = mask.resize((IMG_WIDTH, IMG_HEIGHT), Image.NEAREST)
    return np.array(mask)

def generate_segmentation_mask(model, image_tensor):
    with torch.no_grad():
        output = model(image_tensor)  # shape: (B, NUM_CLASSES, H, W)
        pred = torch.argmax(output, dim=1)
        return pred.squeeze().cpu().numpy()  # shape: (H, W)

def mask_to_rgb(mask_array):
    h, w = mask_array.shape
    rgb_image = np.zeros((h, w, 3), dtype=np.uint8)
    # Define the mapping for 3 classes
    class_rgb_mapping = {
        0: (0, 0, 0),      # Black for background (or class 0)
        1: (0, 255, 0),    # Green for class 1
        2: (255, 0, 0)     # Red for class 2
    }
    for cls, color in class_rgb_mapping.items():
        rgb_image[mask_array == cls] = color
    return rgb_image

# ------------------------------------------------------------------------------
# 5) Main execution: load models, select images, and create merged figure
# ------------------------------------------------------------------------------
if __name__ == "__main__":
    # Load all models
    loaded_models = []
    for model_name, model_path in models_info:
        if not os.path.isfile(model_path):
            continue
        model = load_full_model(model_path)
        loaded_models.append((model_name, model))
    if len(loaded_models) == 0:
        exit(0)
    
    # Gather test images
    test_image_files = [f for f in os.listdir(TEST_IMAGE_FOLDER) if f.lower().endswith(('.jpg','.jpeg','.png'))]
    if len(test_image_files) == 0:
        exit(0)
    
    # Randomly select up to 5 images
    selected_images = random.sample(test_image_files, 5) if len(test_image_files) >= 5 else test_image_files

    # Set up figure:
    n_rows = len(selected_images)
    n_cols = 2 + len(loaded_models)  # 1: Input, 1: GT, rest: each model's prediction
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(4 * n_cols, 4.2 * n_rows))
    if n_rows == 1:
        axes = [axes]  # ensure axes is a list of rows

    # Process each image:
    for row_idx, image_file in enumerate(selected_images):
        input_image_path = os.path.join(TEST_IMAGE_FOLDER, image_file)
        gt_mask_name = os.path.splitext(image_file)[0] + '_morphed.png'
        gt_mask_path = os.path.join(GROUND_TRUTH_MASK_FOLDER, gt_mask_name)

        # Load and resize input image and ground truth mask for display
        original_image = Image.open(input_image_path).resize((IMG_WIDTH, IMG_HEIGHT))
        gt_mask_np = load_ground_truth_mask(gt_mask_path)
        gt_rgb = mask_to_rgb(gt_mask_np)

        # Preprocess image for model inference
        input_tensor = preprocess_image(input_image_path)

        # Column 0: Input image
        axes[row_idx][0].imshow(original_image)
        if row_idx == 0:
            axes[row_idx][0].set_title("Input Image", fontsize=27)
        axes[row_idx][0].axis('off')

        # Column 1: Ground truth mask
        axes[row_idx][1].imshow(gt_rgb)
        if row_idx == 0:
            axes[row_idx][1].set_title("Ground Truth Mask", fontsize=27)
        axes[row_idx][1].axis('off')

        # Next columns: Predictions from each model
        for model_i, (model_name, model_obj) in enumerate(loaded_models):
            pred_mask = generate_segmentation_mask(model_obj, input_tensor)
            pred_rgb = mask_to_rgb(pred_mask)
            col_idx = 2 + model_i
            axes[row_idx][col_idx].imshow(pred_rgb)
            if row_idx == 0:
                axes[row_idx][col_idx].set_title(f"{model_name} \nPredicted Mask", fontsize=27)
            axes[row_idx][col_idx].axis('off')

    plt.tight_layout()
    merged_filename = os.path.join(PREDICTION_SAVE_FOLDER, "All_Models_Predictions.png")
    plt.savefig(merged_filename, bbox_inches='tight',dpi=200)
    # Uncomment the next line if you wish to display the figure interactively
    plt.show()


/tmp/ipykernel_4204/1757816287.py:41: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(model_path, map_location=DEVICE)
/tmp/ipykernel_4204/1757816287.py:144

# Saving Predictions of Each Model

In [14]:
import os
import random
import numpy as np
import torch
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt

# ------------------------------------------------------------------------------
# 1) Automatically gather all directories starting with "Unet-"
# ------------------------------------------------------------------------------
model_dirs = [d for d in os.listdir('.') if os.path.isdir(d) and d.startswith("Unet")]

# ------------------------------------------------------------------------------
# 2) Define directories and hyperparameters
# ------------------------------------------------------------------------------
TEST_IMAGE_FOLDER        = 'CWD-3HSV/test/images/'
GROUND_TRUTH_MASK_FOLDER = 'CWD-3HSV/test/Morphed_Images/'
# (The merged prediction figure for each model will be saved in a "Predictions" subfolder of that model directory.)
IMG_HEIGHT  = 640
IMG_WIDTH   = 640
NUM_CLASSES = 3
DEVICE      = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# ------------------------------------------------------------------------------
# 3) Define the transformation (as used during training)
# ------------------------------------------------------------------------------
transform = transforms.Compose([
    transforms.Resize((IMG_HEIGHT, IMG_WIDTH)),
    transforms.ToTensor(),
])

# ------------------------------------------------------------------------------
# 4) Helper functions
# ------------------------------------------------------------------------------
def load_full_model(model_dir):
    """Load the full model from <model_dir>/unet_best_model.pth."""
    model_path = os.path.join(model_dir, "unet_best_model.pth")
    if not os.path.isfile(model_path):
        return None
    model = torch.load(model_path, map_location=DEVICE)
    model = model.to(DEVICE)
    model.eval()
    return model

def preprocess_image(image_path):
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(DEVICE)
    return image

def load_ground_truth_mask(mask_path):
    mask = Image.open(mask_path).convert('L')
    mask = mask.resize((IMG_WIDTH, IMG_HEIGHT), Image.NEAREST)
    return np.array(mask)

def generate_segmentation_mask(model, image_tensor):
    with torch.no_grad():
        # model outputs logits of shape (B, NUM_CLASSES, H, W)
        output = model(image_tensor)
        pred = torch.argmax(output, dim=1)
        return pred.squeeze().cpu().numpy()  # shape: (H, W)

def mask_to_rgb(mask_array):
    h, w = mask_array.shape
    rgb_image = np.zeros((h, w, 3), dtype=np.uint8)
    # Fixed mapping for 3 classes
    class_rgb_mapping = {
        0: (0, 0, 0),      # Black
        1: (0, 255, 0),    # Green
        2: (255, 0, 0)     # Red
    }
    for cls, color in class_rgb_mapping.items():
        rgb_image[mask_array == cls] = color
    return rgb_image

# ------------------------------------------------------------------------------
# 5) Main execution: Process each model directory
# ------------------------------------------------------------------------------
if __name__ == "__main__":
    # Gather test images (all jpg/jpeg/png)
    test_image_files = [f for f in os.listdir(TEST_IMAGE_FOLDER) if f.lower().endswith(('.jpg','.jpeg','.png'))]
    if len(test_image_files) == 0:
        exit(0)
    
    # Randomly select up to 5 images
    selected_images = random.sample(test_image_files, 5) if len(test_image_files) >= 5 else test_image_files

    # Process each model directory that starts with "Unet-"
    for model_dir in model_dirs:
        model = load_full_model(model_dir)
        if model is None:
            continue
        
        # Create a Predictions subfolder inside the model directory
        predictions_dir = os.path.join(model_dir, "Predictions")
        os.makedirs(predictions_dir, exist_ok=True)
        
        # Set up a figure with one row per image and 3 columns (Input, Ground Truth, Predicted)
        n_rows = len(selected_images)
        n_cols = 3
        fig, axes = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=(4 * n_cols, 4.2 * n_rows))
        if n_rows == 1:
            axes = [axes]  # Ensure axes is a list of rows

        for row_idx, image_file in enumerate(selected_images):
            input_image_path = os.path.join(TEST_IMAGE_FOLDER, image_file)
            gt_mask_name = os.path.splitext(image_file)[0] + '_morphed.png'
            gt_mask_path = os.path.join(GROUND_TRUTH_MASK_FOLDER, gt_mask_name)
            
            # Load original image and ground truth for display
            original_image = Image.open(input_image_path).resize((IMG_WIDTH, IMG_HEIGHT))
            gt_mask_np = load_ground_truth_mask(gt_mask_path)
            gt_rgb = mask_to_rgb(gt_mask_np)
            
            # Preprocess image for inference
            input_tensor = preprocess_image(input_image_path)
            
            # Column 0: Input image
            axes[row_idx][0].imshow(original_image)
            if row_idx == 0:
                axes[row_idx][0].set_title("Input Image", fontsize=27)
            axes[row_idx][0].axis('off')
            
            # Column 1: Ground truth mask
            axes[row_idx][1].imshow(gt_rgb)
            if row_idx == 0:
                axes[row_idx][1].set_title("Ground Truth Mask", fontsize=27)
            axes[row_idx][1].axis('off')
            
            # Column 2: Predicted mask from this model
            pred_mask = generate_segmentation_mask(model, input_tensor)
            pred_rgb = mask_to_rgb(pred_mask)
            axes[row_idx][2].imshow(pred_rgb)
            if row_idx == 0:
                axes[row_idx][2].set_title(f"{model_dir} \nPredicted Mask", fontsize=27)
            axes[row_idx][2].axis('off')
        
        plt.tight_layout()
        merged_filename = os.path.join(predictions_dir, "All_Models_Predictions.png")
        # Save the figure at high resolution
        plt.savefig(merged_filename, bbox_inches='tight', dpi=300)
        plt.close(fig)


/tmp/ipykernel_4204/3991625222.py:41: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(model_path, map_location=DEVICE)


# Saving Results of each model

In [15]:
import os
import pandas as pd

# ------------------------------------------------------------------------------
# 1) Automatically gather all directories starting with "Unet-"
# ------------------------------------------------------------------------------
model_directories = [d for d in os.listdir('.') if os.path.isdir(d) and d.startswith("Unet")]

# Name of the Excel file in each model directory
excel_filename = "Performance_Evaluation_Metrics.xlsx"

# Loop over each found model directory
for model_dir in model_directories:
    # Construct the full path to the Excel file
    excel_path = os.path.join(model_dir, excel_filename)
    
    # Skip this directory if the file does not exist
    if not os.path.isfile(excel_path):
        continue
    
    # Read the Excel file into a DataFrame
    df = pd.read_excel(excel_path)
    if df.empty:
        continue

    # Get the last (bottom) row from the DataFrame
    last_row = df.iloc[-1]
    
    # ------------------ Overall Metrics ------------------
    overall_metrics = {
        "Metric": [
            "Accuracy",
            "Precision",
            "Recall",
            "F1 Score",
            "Mean IoU",
            "Frequency Weighted IoU",
            "Mean Dice",
            "Mean Jaccard"
        ],
        "Value": [
            last_row["Accuracy"],
            last_row["Precision"],
            last_row["Recall"],
            last_row["F1 Score"],
            last_row["Mean IoU"],
            last_row["Frequency Weighted IoU"],
            last_row["Mean Dice"],
            last_row["Mean Jaccard"]
        ]
    }
    overall_df = pd.DataFrame(overall_metrics)
    
    # ------------------ Per-Class Metrics ------------------
    per_class_data = {
        "Class": ["Class 0", "Class 1", "Class 2"],
        "Accuracy": [
            last_row["Accuracy Class 0"],
            last_row["Accuracy Class 1"],
            last_row["Accuracy Class 2"]
        ],
        "Precision": [
            last_row["Precision Class 0"],
            last_row["Precision Class 1"],
            last_row["Precision Class 2"]
        ],
        "Recall": [
            last_row["Recall Class 0"],
            last_row["Recall Class 1"],
            last_row["Recall Class 2"]
        ],
        "F1 Score": [
            last_row["F1 Score Class 0"],
            last_row["F1 Score Class 1"],
            last_row["F1 Score Class 2"]
        ],
        "IoU": [
            last_row["IoU Class 0"],
            last_row["IoU Class 1"],
            last_row["IoU Class 2"]
        ],
        "Dice": [
            last_row["Dice Coefficient Class 0"],
            last_row["Dice Coefficient Class 1"],
            last_row["Dice Coefficient Class 2"]
        ],
        "Jaccard": [
            last_row["Jaccard Index Class 0"],
            last_row["Jaccard Index Class 1"],
            last_row["Jaccard Index Class 2"]
        ]
    }
    per_class_df = pd.DataFrame(per_class_data)
    
    # ------------------------------------------------------------------------------
    # 3) Save the new Excel files in a Results subfolder of the model directory
    # ------------------------------------------------------------------------------
    results_dir = os.path.join(model_dir, "Results")
    os.makedirs(results_dir, exist_ok=True)
    
    overall_excel_path = os.path.join(results_dir, "Overall_Metrics.xlsx")
    per_class_excel_path = os.path.join(results_dir, "Per_Class_Metrics.xlsx")
    
    overall_df.to_excel(overall_excel_path, index=False)
    per_class_df.to_excel(per_class_excel_path, index=False)


# Seperating All Models Perdormance Evaluation Sheet

In [16]:
import os
import pandas as pd

# ------------------------------------------------------------------------------
# 1) Path to the merged Excel file
# ------------------------------------------------------------------------------
merged_excel_path = os.path.join("Results", "All_Models_Performance_Evaluation_Metrics.xlsx")

# ------------------------------------------------------------------------------
# 2) Read the Excel file into a DataFrame
# ------------------------------------------------------------------------------
df = pd.read_excel(merged_excel_path)

# ------------------------------------------------------------------------------
# 3) Mapping to change model names
# ------------------------------------------------------------------------------
name_mapping = {
    "unet_best_model.pth": "Unet",
    "unetplusplus_best_model.pth": "Unet++",
    "manet_best_model.pth": "MAnet",
    "linknet_best_model.pth": "Linknet",
    "fpn_best_model.pth": "FPN",
    "pspnet_best_model.pth": "PSPNet",
    "pan_best_model.pth": "PAN",
    "deeplabv3_best_model.pth": "DeepLabV3",
    "deeplabv3plus_best_model.pth": "DeepLabV3+",
    "upernet_best_model.pth": "UPerNet",
    "segformer_best_model.pth": "Segformer"
}

# Update the "Model Name" column based on the mapping.
# If a model name is not found in the mapping, leave it unchanged.
df["Model Name"] = df["Model Name"].apply(lambda x: name_mapping.get(x, x))

# ------------------------------------------------------------------------------
# 4) Create Overall Metrics DataFrame
# ------------------------------------------------------------------------------
overall_columns = [
    "Model Name",
    "Accuracy",
    "Precision",
    "Recall",
    "F1 Score",
    "Mean IoU",
    "Weighted IoU",
    "Frequency Weighted IoU",
    "Mean Dice",
    "Mean Jaccard"
]
overall_df = df[overall_columns].copy()

# ------------------------------------------------------------------------------
# 5) Create Per-Class Metrics DataFrame
# ------------------------------------------------------------------------------
per_class_columns = [
    "Model Name",
    "Accuracy Class 0", "Accuracy Class 1", "Accuracy Class 2",
    "Precision Class 0", "Precision Class 1", "Precision Class 2",
    "Recall Class 0", "Recall Class 1", "Recall Class 2",
    "F1 Score Class 0", "F1 Score Class 1", "F1 Score Class 2",
    "IoU Class 0", "IoU Class 1", "IoU Class 2",
    "Dice Coefficient Class 0", "Dice Coefficient Class 1", "Dice Coefficient Class 2",
    "Jaccard Index Class 0", "Jaccard Index Class 1", "Jaccard Index Class 2"
]
per_class_df = df[per_class_columns].copy()

# ------------------------------------------------------------------------------
# 6) Save the two DataFrames to Excel (overwrite if re-executed)
# ------------------------------------------------------------------------------
overall_excel_path = os.path.join("Results", "Overall_Metrics.xlsx")
per_class_excel_path = os.path.join("Results", "Per_Class_Metrics.xlsx")

overall_df.to_excel(overall_excel_path, index=False)
per_class_df.to_excel(per_class_excel_path, index=False)

print(f"Saved overall metrics to: {overall_excel_path}")
print(f"Saved per-class metrics to: {per_class_excel_path}")
